In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
import k3d
from ipywidgets import *
from abcd import *
from utils import *

In [3]:
ABCD

abcd.ABCD

In [4]:
def geometry(mir, roc):
    Nm = len(mir)
    M = mir[:,None,:]-mir[None,:,:]
    m = norm(M)
    
    n = norm(np.array([np.cross(m[j,j-1],m[j,(j+1)%Nm]) for j in range(Nm)]))
    refl = norm(np.array([0.5*(m[j,j-1]+m[j,(j+1)%Nm]) for j in range(Nm)])) #vectors normal to reflecting mirrors
    angles = np.array([0.5*np.arccos(np.dot(m[j,j-1],m[j,(j+1)%Nm])) for j in range(Nm)])
    xin = n
    xout = n
    yin = norm(np.array([np.cross(n[j],m[j,j-1]) for j in range(Nm)]))
    yout = norm(np.array([np.cross(n[j],m[j,(j+1)%Nm]) for j in range(Nm)]))
    R = np.stack([np.array([[xout[i]@xin[(i+1)%Nm], yout[i]@xin[(i+1)%Nm]],\
                            [xout[i]@yin[(i+1)%Nm], yout[i]@yin[(i+1)%Nm]]]) for i in range(Nm)], axis=0)
    
    Ls = [np.linalg.norm(M[j-1,j]) for j in range(Nm)]
    Lrt = sum(Ls)
    #Mrt = np.identity(4)
    elements = []
    for i in range(Nm):
        d = np.linalg.norm(M[i-1,i])/2
        #Mrt = Prop(d)@Rot(R[i])@Mirror(roc[i], angles[i])@Prop(d)@Mrt
        elements.append(Prop(d))
        elements.append(Mirror(roc[i], angles[i]))
        elements.append(Rot(R[i]))
        elements.append(Prop(d))
    geom = {'mir': mir, 'Ls': Ls, 'Lrt': Lrt, 'm': m, 'n': n, 'refl': refl, 'yin': yin, 'yout': yout}
    return {'elements': elements, 'geometry': geom}

In [17]:
def plot_geometry(mir, m, n, refl, yin, yout, pf=200., **kwargs):
    Nm = len(mir)
    plot = k3d.plot(camera_auto_fit=True, antialias=True)

    col=0xff0000
    plt_line = k3d.line(pf*mir, shader='mesh', width=0.1, color=col)
    plt_line2 = k3d.line(pf*mir[(-1,0),...], shader='mesh', width=0.1, color=col)
    plot += plt_line
    plot += plt_line2
    plot += k3d.vectors(origins=pf*mir, vectors=n*2, use_head=True, head_size=3.)#Normals = xIn = xOut
    plot += k3d.vectors(origins=pf*mir, vectors=yin*2, use_head=True, head_size=3., color= 0xff8c00) #yIn
    plot += k3d.vectors(origins=pf*mir, vectors=yout*2, use_head=True, head_size=3., color= 0xff8c00) #yOut
    plot += k3d.vectors(origins=pf*mir, vectors=refl*2, use_head=True, head_size=3., color=0x00ff00)

    ey = np.array([0,1,0])
    ex = np.array([1,0,0])
    for i in range(Nm):
        mirror = Box(size=(0.1,.5,.5)).mesh

        mirror.transform.custom_matrix = pad3to4(getrot(ex, refl[i])) #get rotation matrix of mirror
        mirror.transform.translation = pf*mir[i]
        plot += mirror

    return plot

In [6]:
def Tetrahedron(La, thet, roc):
    
    if not isinstance(roc, list):
        roc = [roc]*4
        
    thet = np.deg2rad(thet)
    mir = np.array([(-La/2, La*np.tan(thet), 0),\
                ( La/2, 0, -La*np.tan(thet)),\
                (-La/2, -La*np.tan(thet), 0),\
                ( La/2, 0,  La*np.tan(thet))], dtype=np.float32)
    return {'mir': mir, 'roc': roc}

In [22]:
def SixMirror(dx=27.77, dy=8.0, dz=16.685, d=4.750, dzF=1.5825, Rfast=25.0):

    p5 = np.array([0,0,0])
    p6 = np.array([dx, dy, dzF])
    p1 = np.array([0, dy, dzF])
    p2 = np.array([dx, 2*dy, 0])
    p3 = np.array([d, dy+d, dz])
    p4 = np.array([dx-d, dy-d, dz])
    
    mir = np.stack([p1,p2,p3,p4,p5,p6], axis=0)
    roc = np.array([Rfast, 1e6, 1e6, 1e6, 1e6, Rfast])
    return {'mir': mir, 'roc': roc}

In [23]:
Tetrahedron(La = 5.1e-2, thet = 20, roc=100e-3)['mir'].shape

(4, 3)

In [24]:
#opt = Tetrahedron(La = 5.1e-2, thet = 20, roc=100e-3)
#opt = OriginalTwister(betal=1.35, R=0.025, Rlarge=-0.075, thet=20., asym = 1.25)
opt = SixMirror()
out = geometry(**opt)
geom = out['geometry']
elements = out['elements']

In [25]:
plot_geometry(**geom, pf=1.)

C:\ProgramData\Anaconda3\lib\site-packages\traittypes\traittypes.py:101: UserWarning: Given trait value dtype "float64" does not match required type "float32". A coerced copy has been created.
  np.dtype(self.dtype).name))


Plot(antialias=True, axes=['x', 'y', 'z'], axes_helper=1.0, background_color=16777215, camera=[2, -3, 0.2, 0.0…

In [26]:
system = ABCDSystem(elements)

[-2j, -2j]


In [27]:
system.waist_at(0)

array([0.01938977, 0.01866838])

In [29]:
Npts = 500
lrt = system.Ltot
x = np.linspace(0, lrt, Npts)
#ws = np.zeros_like(x)

ws = system.compute_waists(x)

fig, ax = plt.subplots()
line = ax.plot(x, ws)[0]
ax.set_xlabel(r"$L$ [mm]")
ax.set_ylabel(r"Waist $\omega$ [um]")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'Waist $\\omega$ [um]')